In [13]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import csv

In [6]:
# Read to the correct number in the dataset
ds = load_dataset("gbharti/finance-alpaca", cache_dir="../cache_dir", split=f"train[:24%]")
# If just generating for evaluation, look at eval set vs. gen set
idxs = range(int(0.8) * len(ds), len(ds))
ds = ds.select(idxs)
ds = ds.map(
    lambda example: {"text": 
                    f"<s>[INST] {example['instruction']} [/INST]"},
    num_proc=4)

In [7]:
def generate_example(model, tokenizer, prompt, max_len):
    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_len = tokens.input_ids.shape[1]
    out = model.generate(**tokens, max_length=max_len, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.batch_decode(out[:,prompt_len:], skip_special_tokens=True)
    return response

In [9]:
model_names = ["mistralai/Mistral-7B-v0.1", "../results/sft-finqa/final_model", "../results/dpo-finqa/final_model"]

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        use_nested_quant = True,
    )
model = AutoModelForCausalLM.from_pretrained(model_names[0], 
                                             cache_dir="../cache_dir", 
                                             device_map={"": 0}, 
                                             quantization_config=bnb_config)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_names[0],
                                          cache_dir="../cache_dir")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
field_names = ["prompt", "response"]
with open("base_generations.csv", "w", newline='') as f:
    writer = csv.DictWriter(f, quoting=csv.QUOTE_MINIMAL, fieldnames=field_names)
    writer.writeheader()
    for prompt in tqdm(ds["text"][:8]):
        response = generate_example(model, tokenizer, prompt, 1024)
        writer.writerow({"prompt": prompt, "response": response[0]})

  0%|          | 0/8 [00:00<?, ?it/s]

 38%|███▊      | 3/8 [01:42<02:50, 34.05s/it]


UnicodeEncodeError: 'ascii' codec can't encode character '\u2019' in position 170: ordinal not in range(128)